2. 웹스크래핑 연습문제
2-1. Nate 뉴스기사 제목 스크래핑하기 (필수)
https://news.nate.com/recent?mid=n0100
최신뉴스, 정치 , 경제, 사회, 세계, IT/과학 
6개의 섹션의 뉴스를 출력하는 함수를 생성하여 스크래핑 하기

Image, 기사제목, 기사링크

뉴스기사의 Image를 출력 하세요 
1) Image의 도메인이름이 포함된 경로와 src 속성의 경로를 합치려면 urljoin 함수를 사용하세요.
    from urllib.parse import urljoin
    
    url = ‘https://news.nate.com/recent?mid=n0100’
    src=’ //thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/na/2025/07/23/7408335_high.jpg’

2) Image 출력은 Image 클래스와 display 함수를 사용하세요.
    from IPython.display import Image, display

3) img 엘리먼트의 존재 여부를 체크하신 후에 src 속성의 이미지를 경로를 추출하기
  => Image 가 없는 뉴스도 있기 때문에 



In [3]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Image, display
from urllib.parse import urljoin

section_dict = {'최신뉴스':100,'정치':200,'경제':300,'사회':400,'세계':500,'IT/과학':600}

def print_news(section):
    sid = section_dict[section]
    url = f'https://news.nate.com/section?mid=n0{sid}'
    
    req_header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }

    res=requests.get(url,headers=req_header)
    print(res.status_code, res.ok)
    if res.ok:
        res.encoding = 'utf-8'
        html = res.text
        soup = BeautifulSoup(html, "html.parser")

        news_blocks = soup.select("div.mlt01")

        for idx, block in enumerate(news_blocks,1):     
            a_tag = block.select_one("a.tit")   
            if not a_tag:
                continue
            link = urljoin(url, a_tag['href'])
                
            title_tag = a_tag.select_one("h2.tit")
            title = title_tag.text.strip() if title_tag else "제목 없음"

            # 이미지 (있을 경우만 출력)
            img_tag = block.select_one("img")
            if img_tag and img_tag.get("src"):
                img_url = urljoin(url, img_tag["src"])
            else:
                img_url = None

            # 출력
            print(f"{idx}. {title}")
            print(link)
            if img_url:
                print(f"[이미지]: {img_url}")
                display(Image(url=img_url))
            else:
                print("[이미지 없음]")
            print("-" * 80)


            
    else:
         
        print(f'Error Code = {res.status_code}') 



for section in section_dict:
    print_news(section)

200 True
200 True
200 True
200 True
200 True
200 True


In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

# 섹션 이름과 mid 코드 매핑
section_dict = {
    '최신뉴스': 'n0100',
    '정치': 'n0200',
    '경제': 'n0300',
    '사회': 'n0400',
    '세계': 'n0500',
    'IT/과학': 'n0600'
}

def print_news(section_name):
    mid_code = section_dict[section_name]
    url = f"https://news.nate.com/recent?mid={mid_code}"
    
    print(f"\n======> {url} ({section_name}) <======")

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/104.0.0.0 Safari/537.36"
    }

    res = requests.get(url, headers=headers)
    
    if res.ok:
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, 'html.parser')

        # 뉴스 블록들 찾기
        news_blocks = soup.select("div.mlt01")

        for idx, block in enumerate(news_blocks, 1):
            # 링크
            a_tag = block.select_one("a.tit")
            if not a_tag:
                continue
            link = urljoin(url, a_tag['href'])

            # 제목
            title_tag = a_tag.select_one("h2.tit")
            title = title_tag.text.strip() if title_tag else "제목 없음"

            # 이미지 (있을 경우만 출력)
            img_tag = block.select_one("img")
            if img_tag and img_tag.get("src"):
                img_url = urljoin(url, img_tag["src"])
            else:
                img_url = None

            # 출력
            print(f"{idx}. {title}")
            print(link)
            if img_url:
                print(f"[이미지]: {img_url}")
                display(Image(url=img_url))
            else:
                print("[이미지 없음]")
            print("-" * 80)

    else:
        print(f"요청 실패: {res.status_code}")

# ✅ 섹션별로 호출
for section in section_dict:
    print_news(section)



======> https://news.nate.com/recent?mid=n0100 (최신뉴스) <======

======> https://news.nate.com/recent?mid=n0200 (정치) <======

======> https://news.nate.com/recent?mid=n0300 (경제) <======

======> https://news.nate.com/recent?mid=n0400 (사회) <======

======> https://news.nate.com/recent?mid=n0500 (세계) <======

======> https://news.nate.com/recent?mid=n0600 (IT/과학) <======


2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기 (필수)
:  하나의 웹툰의 제목(title)과 회차번호(no),회차의URL(url) 을 입력으로 받는 함수를 선언합니다. 
   def download_one_episode(title,no,url):

아래와 같이 호출합니다.
download_one_episode('일렉시드',341,'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed')

img\일렉시드\341 디렉토리가 생성되며 , 
그 디렉토리 아래에 웹툰 image들이 다운로드 되도록 해주세요.
